# Audio Recognition
### Tim Nguyen & Neeraj Shah
---


In the following code portions below, we implement a barebones version of how audio recognition works using the core concept sof digital signal processing.  This involves sampling the analog signal (sound input), converting it a digital signal, performing a discrete fourier transform, and matching the audio fingerprints with a predefined database.

### Dependencies

1) Need to have PyAudio library.  Install with python's package manager `pip` :

`pip install pyaudio`



In [19]:
# Import required libraries

from sys import byteorder
from array import array
import time
import copy
import pyaudio
import wave
import numpy as np
import matplotlib.pyplot as plt
import scipy
import scipy.io


In [17]:
# Sample the songs in the library to create the TRUE database.



## Code below deals with audio input

Records audio and saves to a .WAV file for a specified amount of seconds.

In [11]:
def record(fileName):
    
    CHUNK = 1024
    FORMAT = pyaudio.paInt16
    CHANNELS = 1
    RATE = 44100  # Sampling Rate
    RECORD_SECONDS = 5
    WAVE_OUTPUT_FILENAME = fileName

    p = pyaudio.PyAudio()

    stream = p.open(format=FORMAT,
                    channels=CHANNELS,
                    rate=RATE,
                    input=True,
                    frames_per_buffer=CHUNK)

    print("Recording Audio ... ")

    frames = []

    for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK)
        frames.append(data)

    print("Recording Finished")

    stream.stop_stream()
    stream.close()
    p.terminate()

    wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))
    wf.close()
    
def play(fileName):

    wf = wave.open(fileName, 'rb')

    p = pyaudio.PyAudio()

    def callback(in_data, frame_count, time_info, status):
        data = wf.readframes(frame_count)
        return (data, pyaudio.paContinue)

    stream = p.open(format=p.get_format_from_width(wf.getsampwidth()),
                    channels=wf.getnchannels(),
                    rate=wf.getframerate(),
                    output=True,
                    stream_callback=callback)

    stream.start_stream()

    i = 0
    # while stream.is_active():
    while i < 125:
        time.sleep(0.1)
        i += 1

    stream.stop_stream()
    stream.close()
    wf.close()

    p.terminate()

In [23]:
record("test.wav")  # Test record

* recording
* done recording


In [12]:
play("billiejeanwav.wav")

In [21]:
sample_rate, X = scipy.io.wavfile.read('billiejeanwav.wav')
print (sample_rate, X.shape )
plt.specgram(X[:,0], Fs=sample_rate, xextent=(0,30))




#plt.pcolormesh(times, frequencies, spectogram)
#plt.imshow(spectogram)
#plt.ylabel('Frequency [Hz]')
#plt.xlabel('Time [sec]')
#plt.show()

(44100, (12973871, 2))


/usr/local/lib/python2.7/site-packages/matplotlib/axes/_axes.py:7222: RuntimeWarning: divide by zero encountered in log10
  Z = 10. * np.log10(spec)


(array([[  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
           2.22392700e-07,   4.02813114e-06,   4.17905738e-06],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
           1.45428763e-06,   4.37183538e-06,   2.20481872e-06],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
           1.19073358e-06,   7.66671975e-07,   1.53443377e-06],
        ..., 
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
           7.34012043e-06,   1.21088359e-06,   4.14202951e-06],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
           1.05143587e-05,   3.45534693e-06,   1.24720335e-06],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
           4.60010705e-06,   4.77152239e-06,   1.38953785e-10]]),
 array([     0.      ,    172.265625,    344.53125 ,    516.796875,
           689.0625  ,    861.328125,   1033.59375 ,   1205.859375,
          1378.125   ,   1550.390625,   1722.65625 ,   1